In [1]:
import pandas as pd
from sqlalchemy import create_engine
import string

In [2]:
engine = create_engine("postgresql://localhost/mimic?user=xxxxx&password=xxxxx") 
# user name and password needs to be modified based on the local setup of the postgress server

In [3]:
df2 = pd.read_sql_query(
    'select hadm_id, text from mimic.mimiciii.noteevents where category=\'Discharge summary\'', con=engine)  # change limit number
df2 = df2.sort_values('hadm_id').reset_index()
df = df2.groupby('hadm_id').agg(lambda x: ''.join(x.unique()))
df = df.reset_index()

In [4]:
df.head()

,hadm_id,text
0,100001,Admission Date: [**2117-9-11**] ...
1,100003,Admission Date: [**2150-4-17**] ...
2,100006,Admission Date: [**2108-4-6**] Discharg...
3,100007,Admission Date: [**2145-3-31**] ...
4,100009,Admission Date: [**2162-5-16**] ...


In [6]:
import re, nltk
def preprocessing(t):
    t = re.sub('\n', ' ',t)
    t = re.sub('\r', ' ',t)
    t = t.lower()
    t = re.sub('\\[(.*?)\\]', '', t)  # remove de-identified brackets
    t = re.sub('[0-9]+\.', '', t)  
    t = re.sub('dr\.', 'doctor', t)
    t = re.sub('m\.d\.', 'md', t)
    t = re.sub('admission date:', '', t)
    t = re.sub('discharge date:', '', t)
    t = re.sub('--|__|==', '', t)
    return t

max_size = 1500
def truncate(content, length=max_size):
    word = content.split()
    if len(word) <= length:
        return content
    else:
        return ' '.join(word[:length])

In [12]:
hname = "fulltext_preprocessed.csv"

with open(hname, "a") as g:
    g.write('hadm,text\n')
    
with open(hname, "a") as h:
    for index, line in df.iterrows():
        t = line[1]
        tfinal = preprocessing(t)
        text = "".join([w for w in tfinal if w not in string.punctuation])
        t2 = text.lower()
        ttrial = t2.split()
        tt = [y for y in ttrial if not y.isnumeric()]
        tf = " ".join(tt)
        ttr = truncate(tf)
        h.write(str(line[0])+','+str(ttr)+'\n')
        

In [15]:
processedtext = pd.read_csv("fulltext_preprocessed.csv")
processedtext.head()

,hadm,text
0,100001,date of birth sex f service medicine allergies...
1,100003,date of birth sex m service medicine allergies...
2,100006,date of birth sex f service o med chief compla...
3,100007,date of birth sex f service surgery allergies ...
4,100009,date of birth sex m service cardiothoracic all...
